In [1]:
import pandas as pd
import pickle
import numpy as np
import re
import spacy
import string
string.punctuation += '➡•’…'



In [2]:
dem_df = pickle.load(open('../../../../data/all_tweet_texts_dem.p', 'rb'))

In [3]:
rep_df = pickle.load(open('../../../../data/all_tweet_texts_rep.p', 'rb'))

In [4]:
len(dem_df), len(rep_df)

(134337, 85456)

In [5]:
dem_text = dem_df.Text.tolist()
rep_text = rep_df.Text.tolist()


In [6]:
text = dem_text + rep_text

In [7]:
text[0]

"We had our own bad weather &amp; power outages here today, but we're thinking of our friends down south &amp; who were hit hard by devastating tornadoes &amp; extreme weather. We are with you in spirit, we will pitch in to help you recover, &amp; we are all #OneNation."

In [8]:
# # without processing
# with open('glove_corpus_basic.txt', 'w') as g:
#     for line in text:
#         g.write(line)
#         g.write('\n')

In [9]:
# vocab = open('vocab_basic.txt', 'r').readlines()
# covid_vocab = []
# for line in vocab:
#     line = line.strip().lower()
#     if 'covid' in line or 'corona' in line:
#         covid_vocab.append(line.split(' ')[0])

In [10]:
# pickle.dump(covid_vocab, open('../../../../data/all_covid_words.p', 'wb'))

In [8]:
covid_vocab = pickle.load(open('../../../../data/all_covid_words.p', 'rb'))

In [9]:
# covid_vocab

In [10]:
big_regex = re.compile('|'.join(map(re.escape, covid_vocab)))

In [11]:
processed_text = []

In [12]:
for line in text:
    line = line.lower()
    processed_line = big_regex.sub('covid', line)
    processed_text.append(processed_line)

In [13]:
len(processed_text)

219793

In [14]:
nlp=spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner'])

In [15]:
processed_text[0]

"we had our own bad weather &amp; power outages here today, but we're thinking of our friends down south &amp; who were hit hard by devastating tornadoes &amp; extreme weather. we are with you in spirit, we will pitch in to help you recover, &amp; we are all #onenation."

In [16]:
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~➡•’…'

In [17]:
tokenized_text = []
for line in processed_text:
    line = line.replace('&amp;', '&')
    line = line.translate(str.maketrans('', '', string.punctuation)).replace('\n', '') 
    tokenized_line = [token.text for token in nlp(line)]
    tokenized_text.append(' '.join(tokenized_line))


In [18]:
# sample = "We're grateful to so many essential workers whose jobs have taken on added #covidpandemic risk during this pandemic."

In [20]:
# big_regex.sub('covid', sample)

In [21]:
for line in tokenized_text[0:5]:
    print(line)

we had our own bad weather   power outages here today but were thinking of our friends down south   who were hit hard by devastating tornadoes   extreme weather we are with you in spirit we will pitch in to help you recover   we are all onenation
rt agneronha questions about scams price gouging background checks public meetings   records requestsvisit our resource page for in
ris agneronha is leading efforts to stop scammers if you see a covidscam or think you have been the victim of a scam you may contact the ags consumer protection unit at 401 2744400 or e mail consumersriagrigov
help stop covid scammers if anyone calls or texts you claiming to the be the irs and asks for your personal information regarding your covid direct payment or rebate check do not reply they are not legit
were grateful to so many essential workers whose jobs have taken on added risk during this pandemic we should do more than applaud we must ensure they are fairly compensated heroesfund hazard pay would give 

In [22]:
with open('glove_new_corpus.txt', 'w') as g:
    for line in tokenized_text:
        g.write(line)
        g.write('\n')

## Parse fine tuned vectors

In [2]:
from gensim.test.utils import get_tmpfile, datapath
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

In [3]:
glove2word2vec('vectors_50.txt', './vectors_gensim_word2vec_50_epochs.txt')

(28696, 300)

In [8]:
# Creating a KeyedVectors from a temporary file
model = KeyedVectors.load_word2vec_format('vectors_gensim_word2vec_50_epochs.txt')

In [10]:
model.wv.most_similar('covid')

/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('pandemic', 0.6540495157241821),
 ('outbreak', 0.6431459188461304),
 ('response', 0.6113077402114868),
 ('spread', 0.5741133093833923),
 ('virus', 0.5205848217010498),
 ('crisis', 0.5145406723022461),
 ('now', 0.49959105253219604),
 ('help', 0.4982158839702606),
 ('the', 0.4952136278152466),
 ('health', 0.4913753271102905)]

In [13]:
original_vec = open('glove.6B.300d.txt', 'r').readlines()
original_vec = [line.strip() for line in original_vec]

In [14]:
len(original_vec)

400000

In [15]:
trained_vec = open('vectors_50.txt', 'r').readlines()
trained_vec = [line.strip() for line in trained_vec]

In [16]:
trained_vocab = set([line.split()[0] for line in trained_vec])

In [17]:
with open('all_glove_50_epoch.txt', 'w') as g:
    for line in trained_vec:
        if len(line.split()) == 301:
            g.write(line)
            g.write('\n')
    for line in original_vec:
        key = line.split()[0]
        if key not in trained_vocab:
            g.write(line)
            g.write('\n')

In [18]:
glove2word2vec('all_glove_50_epoch.txt', './all_glove_gensim_word2vec_50_epoch.txt')

(409003, 300)